In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont,ImageColor

class SmartwatchGenerator:
    def __init__(self, size=(224, 224)):
        self.size = size
        self.center = (size[0]//2, size[1]//2)
        self.radius = int(min(size)*0.4)
        self.font = ImageFont.load_default()
        self.image=None
    def draw_watch_face(self, draw, material_type, 
                   material_color=(128,128,128),
                   grid_spacing=5,
                   material_alpha=255):
        """
        绘制智能手表表盘主体，支持完整颜色控制
        
        参数:
        material_type (str): 材质类型('mesh','noise','pure')
        face_color (str/tuple): 纯色材质颜色
        mesh_color (tuple): 网格线条颜色（RGB）
        noise_color (tuple): 噪点基色（RGB）
        grid_spacing (int): 网格线间隔像素
        material_alpha (int): 整体透明度(0-255)
        """
        # 创建圆形遮罩
        mask = Image.new('L', self.size, 0)
        mask_draw = ImageDraw.Draw(mask)
        mask_draw.ellipse([self.center[0]-self.radius, self.center[1]-self.radius,
                          self.center[0]+self.radius, self.center[1]+self.radius], 
                          fill=255)
        
        # 生成材质纹理
        material = self.create_material(material_type, 
                                       face_color=material_color,
                                       mesh_color=material_color,
                                       noise_color=material_color,
                                       grid_spacing=grid_spacing)
        
        # 处理透明度
        material = material.convert('RGBA')
        material.putalpha(mask)
        r, g, b, a = material.split()
        new_alpha = a.point( lambda x: int(x * material_alpha/255))
        material.putalpha(new_alpha)  # 直接应用新Alpha
        return material
    
    def create_material(self, material_type, **kwargs):
        """增强版材质生成器，支持颜色参数"""
        if material_type == 'mesh':
            # 网格材质实现
            grid_color = kwargs.get('mesh_color', (0,0,0))
            grid_color = grid_color if isinstance(grid_color, tuple) else ImageColor.getrgb(grid_color)
            grid_spacing = kwargs.get('grid_spacing', 5)
            
            # 创建透明背景
            arr = np.ones((self.size[1], self.size[0], 4), dtype=np.uint8)*255
            # 绘制水平线
            arr[::grid_spacing, :, :3] = grid_color
            arr[:, ::grid_spacing, :3] = grid_color
            return Image.fromarray(arr)
        
        elif material_type == 'noise':
            # 彩色噪点材质
            base_color = kwargs.get('noise_color', (128,128,128))
            base_color = base_color if isinstance(base_color, tuple) else ImageColor.getrgb(base_color)
            noise = np.ones((self.size[1], self.size[0], 4), dtype=np.uint8)*255
            # 生成彩色噪点
            noise[:, :, :3] = np.random.randint(0, 20, 
                                               (self.size[1], self.size[0], 3), 
                                               dtype=np.uint8) + base_color
            return Image.fromarray(noise.clip(0,255))
        
        else:  # pure
            # 纯色材质
            color = kwargs.get('face_color', (128,128,128))
            color = color if isinstance(color, tuple) else ImageColor.getrgb(color)
            
            
            return Image.new('RGBA', self.size, (*color, 255))

    

    def add_clock_hands(self, draw, hour, minute, second, colors):
        angles = [
            (hour % 12 + minute/60) * 30 - 90,  # Hour hand
            minute * 6 - 90,                    # Minute hand
            second * 6 - 90                     # Second hand
        ]
        
        lengths = [
            self.radius * 0.5,
            self.radius * 0.7,
            self.radius * 0.9
        ]
        
        for i in range(3):
            end_x = self.center[0] + lengths[i] * np.cos(np.radians(angles[i]))
            end_y = self.center[1] + lengths[i] * np.sin(np.radians(angles[i]))
            draw.line([self.center[0], self.center[1], end_x, end_y], 
                     fill=colors[i], width=6-i*2)

   

    def add_weather(self, draw, weather, position, icon_size=30):
        """
        使用预渲染的PNG图标显示天气
        :param weather: 天气类型（对应文件名，如'sunny'）
        :param position: 显示位置（'top-left'等）
        :param icon_size: 图标目标尺寸
        """
       
        # 加载天气图标（假设与脚本同目录）
        icon_path = f"./{weather}.png"
        weather_icon = Image.open(icon_path)
        
        # 缩放图标到指定尺寸
        weather_icon = weather_icon.resize((icon_size, icon_size))
        
        # 计算粘贴位置
        x, y = self.get_corner_position(position)
        # 粘贴到主画布
        self.image.paste(weather_icon, (x, y), weather_icon)
        
                             
    def add_battery(self, draw, percentage, position):
        x, y = self.get_corner_position(position)
        draw.rectangle([x, y, x+25, y+10], outline='black', width=1)
        fill_width = int(23 * percentage/100)
        draw.rectangle([x+2, y+2, x+2+fill_width, y+8], fill='green')
        draw.text((x+30, y), f"{percentage}%", fill='black', font=self.font)

    def get_corner_position(self, position):
        margin = 10
        if position == 'top-left':
            return (margin, margin)
        elif position == 'top-right':
            return (self.size[0]-50, margin)
        elif position == 'bottom-left':
            return (margin, self.size[1]-30)
        else:  # bottom-right
            return (self.size[0]-50, self.size[1]-30)

    def generate(self, hour=12, minute=0, second=0,
                hour_color=(0,0,0), minute_color=(100,100,100), second_color=(255,0,0),
                material='mesh',material_color='#FFD700',material_alpha=50, weather='sunny', battery=100,
                weather_pos='top-left', battery_pos='bottom-right'):
        
        # Create base image
        base = Image.new('RGBA', self.size, (255,255,255,255))
        self.image=base
        draw = ImageDraw.Draw(base)
        
        # Draw watch face with material
        watch_face = self.draw_watch_face(draw, material, 
                                 material_color=material_color, 
                                 material_alpha=material_alpha)
        # 明确指定Alpha通道作为掩码
        alpha_mask = watch_face.getchannel('A')
    
        base.paste(watch_face, (0,0), mask=alpha_mask)
        
        # Draw clock elements
        self.add_clock_hands(draw, hour, minute, second, 
                            [hour_color, minute_color, second_color])
        
        # Add information displays
        self.add_weather(draw, weather, weather_pos)
        self.add_battery(draw, battery, battery_pos)
        
        return base.convert('RGB')



In [ ]:
def quantize_color(rgb):
    color_map = {
        'R': 'Dark' if rgb[0] < 64 else 'Medium' if rgb[0] < 128 else 'Light' if rgb[0] < 192 else 'Bright',
        'G': 'Dark' if rgb[1] < 64 else 'Medium' if rgb[1] < 128 else 'Light' if rgb[1] < 192 else 'Bright',
        'B': 'Dark' if rgb[2] < 64 else 'Medium' if rgb[2] < 128 else 'Light' if rgb[2] < 192 else 'Bright'
    }
    return f"{color_map['R']}-{color_map['G']}-{color_map['B']}"


In [ ]:
import random

# What is 模板 (10个/属性)
WHAT_TEMPLATES = {
    'color': [
        "What is the color of the {}?",
        "Can you tell the {} color?",
        "What color is the {}?",
        "Identify the {} color.",
        "Describe the {}'s color.",
        "What hue is used for the {}?",
        "What shade is the {}?",
        "What is the {}'s color scheme?",
        "What color tone does the {} have?",
        "What is the {} colored?"
    ],
    'weather': [
        "What weather condition is shown?",
        "What is the current weather?",
        "Describe the weather displayed.",
        "What climate is indicated?",
        "What meteorological condition is visible?",
        "What is the atmospheric condition?",
        "What type of weather is depicted?",
        "Identify the weather situation.",
        "What is the weather forecast showing?",
    ],
    'texture': [
        "What material texture is used in the watch face?",
        "Describe the watch face texture.",
        "What surface pattern is visible in the watch face?",
        "What is the texture type of the watch face?",
        "Identify the material pattern of the watch face.",
        "What design texture is applied on the watch face?",
        "What is the watch face surface finish?",
        "What kind of texture is shown on the watch face?",
        "What material effect is used on the watch face?",
        "What is the pattern style on the watch face?"
    ],
    'time': [
        "What is the current time?",
        "What time does the watch show?",
        "What hour/minute/second is displayed?",
        "What's the exact time shown?",
        "What is the time configuration?",
        "What time is indicated?",
        "What is the time reading?",
        "What time is displayed?",
        "What is the watch showing?",
        "What's the time value?"
    ],
    'battery': [
        "What is the battery percentage?",
        "What battery level is shown?",
        "What is the power status?",
        "What's the remaining battery?",
        "What is the battery capacity?",
        "What percentage is displayed?",
        "What is the battery status?",
        "How much battery is left?",
        "What is the charge level?",
        "What's the battery indicator showing?"
    ]
}

# Where is 模板 (10个/属性)
WHERE_TEMPLATES = {
    'battery_pos': [
        "Where is the battery indicator located?",
        "Where is the battery displayed?",
        "What corner shows the battery?",
        "Where is the power status shown?",
        "Where to find the battery info?",
        "Where is the charge level visible?",
        "Where is the battery percentage placed?",
        "In which corner is the battery?",
        "Where is the battery positioned?",
        "Where can you see the battery level?"
    ],
    'weather_pos': [
        "Where is the weather displayed?",
        "Where is the climate info shown?",
        "What corner has the weather?",
        "Where is the meteorological data?",
        "Where to find the weather condition?",
        "Where is the precipitation info?",
        "Where is the atmospheric data?",
        "In which corner is the weather?",
        "Where is the forecast positioned?",
        "Where can you see the weather icon?"
    ]
}

In [ ]:
class VQAGenerator:
    def __init__(self, watch_params):
        self.params = watch_params
        
    def generate_qa_pair(self):
        # 随机选择问题类型
        question_type = random.choice(['what', 'what', 'what', 'where','where'])
        
        # 根据类型选择属性
        if question_type == 'what':
            attribute = random.choice([
                'weather', 'time', 'battery'
            ])
            q_template=random.choice(WHAT_TEMPLATES[attribute])
        else:
            attribute = random.choice(['battery_pos', 'weather_pos'])
            q_template=random.choice(WHERE_TEMPLATES[attribute])
        
        # 获取属性值
        value = self.get_attribute_value(attribute)

        # 特殊处理颜色属性
        if attribute == 'color':
            element = random.choice(['hour hand', 'minute hand', 
                                    'second hand', 'watch face'])
            color = self.get_color_value(element)
            answer = quantize_color(color)
            q_template = q_template.format(element)
        else:
            answer = value
        
        return q_template, answer

    def get_attribute_value(self, attribute):
        mapping = {
            'weather': self.params['weather'],
            'texture': self.params['material'],
            'time': f"{self.params['hour']:02}:{self.params['minute']:02}:{self.params['second']:02}",
            'battery': f"{self.params['battery']}%",
            'weather_pos': self.params['weather_pos'],
            'battery_pos': self.params['battery_pos']
        }
        return mapping.get(attribute, None)
    
    def get_color_value(self, element):
        color_map = {
            'hour hand': self.params['hour_color'],
            'minute hand': self.params['minute_color'],
            'second hand': self.params['second_color'],
            'watch face': self.params['material_color']
        }
        return color_map[element]

In [ ]:
CAPTION_ATTRIBUTES = {
    # 核心属性（必现）
    'time': 1,
    
    # 颜色属性
    'hour_color': 0,
    'minute_color': 0,
    'second_color': 0,
    
    # 材质属性
    'material': 0,
    
    # 状态属性
    'weather': 0.5,
    'battery': 0.5,
    
    # 位置属性
    'weather_pos': 0.5,
    'battery_pos': 0.5
}

In [ ]:
CAPTION_TEMPLATES = {
    # 时间模板
    'time': [
        ", on which the current time is {}",
        ", displaying {}",
        ", showing the time {}",
        ", on which the time reads {}"
    ],
    
    # 颜色模板
    'hour_color': [
        ", with an hour hand colored {}",
        ", featuring {} hour hand",
        ", with hour hand in {}"
    ],
    'minute_color': [
        ", with minute hand in {}",
        ", with {} minute hand",
        ", featuring {} minute pointer"
    ],
    'second_color': [
        ", with second hand colored {}",
        ", with a {} second pointer",
        ", featuring {} second hand"
    ],
    
    # 材质模板
    'material': [
        ", on a {} watch face",
        ", with {} textured background",
        ", featuring {} surface"
    ],
    
    # 状态模板
    'weather': [
        ", showing {} weather",
        ", with {} weather displayed",
        ", indicating {} conditions"
    ],
    'battery': [
        ", with {}% battery",
        ", showing {}% charge",
        ", displaying {}% power level"
    ],
    
    # 位置模板
    'weather_pos': [
        ", with weather displayed at {}",
        ", with weather indicator at {}",
        ", with weather shown in the {} corner"
    ],
    'battery_pos': [
        ", with battery level at {}",
        ", with battery indicator at {}",
        ", power status displayed in {}"
    ],
    'caption_a':[
        "The image shows a smart watch",
        "There is a watch on the image",
        "A smart watch is show in the image"
    ],
    'generation_q':[
        "Can you generate a image of a smart watch",
        "Can you draw a smart watch",
        "Please show a smart watch"
    ],

    'caption_q':[
        "<image>\nWhat are the key components and features of this image?",
        "<image>\nCould you outline the primary elements of this watch?",
        "<image>\nWhat main features define the appearance of this user interface?",
        "<image>\nPlease identify the central elements and characteristics of this watch design.",
        "<image>\nHow would you describe the core features of this image's visual layout?",
        "<image>\nWhat are the standout elements in this smartwatch interface's design?",
        "<image>\nCould you detail the principal components of this watch face's structure?",
        "<image>\nWhat visual features are most prominent in this image?",
        "<image>\nPlease list the main aspects that make up this user interface.",
        "<image>\nHow would you characterize the key elements of this watch's user interface?"
    ],
    'generation_a':[
        "Sure, here is the image. \n<image>",
        "The image is shown as you wish. \n<image>",
        "The image is generated. \n<image>"
    ]
}

In [ ]:

def generate_caption(params,base, attributes=CAPTION_ATTRIBUTES):
    selected = []
    for key, prob in attributes.items():
        if random.random() < prob:
            selected.append(key)
    # 确保不空
    if len(selected)==0:
        selected.append('time')
    
    # 添加其他属性
    for attr in selected:
        template = random.choice(CAPTION_TEMPLATES[attr])
        if attr == 'time':
            value=f"{params['hour']:02}:{params['minute']:02}:{params['second']:02}"
        elif attr == 'hour_color':
            value = quantize_color(params['hour_color'])
        elif attr == 'minute_color':
            value = quantize_color(params['minute_color'])
        elif attr == 'second_color':
            value = quantize_color(params['second_color'])
        elif attr== 'material':
            value=quantize_color(params['material_color'])
            if random.random() < 0.5:
                value+=f" {params['material']}"
        else:
            value = params[attr]
        
        # 替换占位符
        formatted = template.format(value)
        base += formatted
    
    # 优化句子结构
    base = base.replace("  ", " ").capitalize()
    if base.endswith(','):
        base = base[:-1]
    return base + '.' if not base.endswith('.') else base

def generate_caption_QA(params,mode):
    Q=None
    A=None
    if mode=='caption':
        Q=random.choice(CAPTION_TEMPLATES['caption_q'])
        A=generate_caption(params,random.choice(CAPTION_TEMPLATES['caption_a']))
    elif mode=='generation':
        Q=generate_caption(params,random.choice(CAPTION_TEMPLATES['generation_q']))
        A=random.choice(CAPTION_TEMPLATES['generation_a'])
    return Q, A

In [ ]:
import json
import os
from tqdm import tqdm

# set the data path and number of samples here
image_folder = '.../smart_watch_image_train'
data_file_path= '.../smart_watch_train.json'
num_of_samples = 60000

os.makedirs(image_folder, exist_ok=True)
generator = SmartwatchGenerator()
samples=[]
for index in tqdm(range(num_of_samples)):
    watch_params={}
    watch_params['hour'] = random.randint(0, 12)
    watch_params['minute'] = random.randint(0, 59)
    watch_params['second'] = random.randint(0, 59)
    # watch_params['hour_color'] = tuple(random.randint(0, 255) for _ in range(3))
    # watch_params['minute_color'] = tuple(random.randint(0, 255) for _ in range(3))
    # watch_params['second_color'] = tuple(random.randint(0, 255) for _ in range(3))
    #watch_params['material'] = random.choice(['mesh', 'noise', 'pure'])
    watch_params['material_color'] = tuple(random.randint(0, 255) for _ in range(3))
    watch_params['hour_color'] = (255,255,255)
    watch_params['minute_color'] = (255,255,255)
    watch_params['second_color'] = (255,255,255)
    watch_params['material'] = 'pure'
    # watch_params['material_color'] = (0,0,0)
    watch_params['weather'] = random.choice(['sunny', 'cloudy', 'raining'])
    watch_params['battery'] = random.randint(0, 100)
    pos_list= ['top-left', 'top-right', 'bottom-left', 'bottom-right']
    pos=random.sample(pos_list,2)
    watch_params['weather_pos'] = pos[0]
    watch_params['battery_pos'] = pos[1]


    # image generation -----------------------------------
    vqa_gen = VQAGenerator(watch_params)
    image = generator.generate(
        hour=watch_params['hour'], minute=watch_params['minute'], second=watch_params['second'],
        hour_color=watch_params['hour_color'], minute_color=watch_params['minute_color'], second_color=watch_params['second_color'],
        material=watch_params['material'], material_color=watch_params['material_color'], material_alpha=255,
        weather=watch_params['weather'], battery=watch_params['battery'],
        weather_pos=watch_params['weather_pos'], battery_pos=watch_params['battery_pos']
    )
    image_file_path=f"{index}.png"
    full_image_file_path=f"{image_folder}/{image_file_path}"
    image.save(full_image_file_path,format='PNG')


    # generation data generation -----------------------------------
    Gen_data={
        "task": "generation",
        "image": image_file_path,
        "conversations":[]
    }
    q,a=generate_caption_QA(watch_params,'generation')
    Gen_data['conversations'].append({"from": "human", "value": q})
    Gen_data['conversations'].append({"from": "gpt", "value": a})
    samples.append(Gen_data)


    # vqa data generation -----------------------------------
    VQA_data={
        "task": "vqa",
        "image": image_file_path,
        "conversations":[]
    }
    q, a = vqa_gen.generate_qa_pair()
    VQA_data['conversations'].append({"from": "human", "value": q})
    VQA_data['conversations'].append({"from": "gpt", "value": a})
    for i in range(3):
        if random.random() < 0.4:
            q, a = vqa_gen.generate_qa_pair()
            VQA_data['conversations'].append({"from": "human", "value": q})
            VQA_data['conversations'].append({"from": "gpt", "value": a})
    VQA_data['conversations'][0]['value']='<image>\n'+VQA_data['conversations'][0]['value']
    samples.append(VQA_data)

    # caption data generation -----------------------------------
    Caption_data={
        "task": "caption",
        "image": image_file_path,
        "conversations":[]
    }
    q,a=generate_caption_QA(watch_params,'caption')
    Caption_data['conversations'].append({"from": "human", "value": q})
    Caption_data['conversations'].append({"from": "gpt", "value": a})
    samples.append(Caption_data)


with open(data_file_path, "w") as f:
    json.dump(samples, f, indent=4)